<html>
<div style="width:100%">
  <div style="width:90%; float:left; background:white; margin:10px">
    <img style ="width:100%"src ="https://raw.githubusercontent.com/publicunpublic/QCMM_Jupyter/main/full.svg">
    <div style="margin-top:2.5%;position:absolute;background-color:#00000;width:6%;height:0.3%"></div>
  </div>
  </div>


<html>
<center>
<br>
<h1 style="color:#E6610F;"> Analyze molecules from site sampling </h1>
<br><br>
</center>

In [2]:
import qcportal as ptl

In [1]:
client_address = "152.74.10.245:7778"
username = None 
password = None 
sampling_collection_name = 'salen-OMe+d-lactide' #"h2co_W22_11_0070+hnc"

In [22]:
client = ptl.FractalClient(address=client_address, username = username, password = password, verify=False)

##  A) List structures in the scan collection

In [23]:
ds_opt  = client.get_collection("OptimizationDataset", sampling_collection_name)
ds_opt.status(collapse = False)

,pbe0-d3_def2-svp
salen-OMe+d-lactide_0002,COMPLETE
salen-OMe+d-lactide_0003,COMPLETE
salen-OMe+d-lactide_0004,COMPLETE
salen-OMe+d-lactide_0005,COMPLETE
salen-OMe+d-lactide_0006,COMPLETE
salen-OMe+d-lactide_0007,COMPLETE
salen-OMe+d-lactide_0009,COMPLETE
salen-OMe+d-lactide_0010,COMPLETE
salen-OMe+d-lactide_0011,COMPLETE
salen-OMe+d-lactide_0012,COMPLETE


##  B)  Visualize the molecules

In [44]:
entry_name = sampling_collection_name +"_0025"
model_chem = "pbe0-d3_def2-svp"

rr = ds_opt.get_record(entry_name, specification = model_chem)
mol = rr.get_final_molecule()
mol

NGLWidget()

## C)  Retrive molecule ID  (For QCScan)

In [45]:
mol.id

'39083'

# Extras

### 1) Optimize at different level of theory 

In [ ]:
#kw = ptl.models.KeywordSet(**{"values": {"dftd": "d3", "convthre" : '3.0e-7', "threall" : '1.0e-13', 'dftgrid' : 2, "scf" : "diis+a"}})
#kw_id = client.add_keywords([kw])[0]
kw_id = None

add_spec = {'name':"pbe0-d3bj_def2-svp",
        'description': 'Geometric + Psi4/pbe0-d3bj/def2-svp.',
        'optimization_spec': {'program': 'geometric', 'keywords': None },
        'qc_spec': {'driver': 'gradient',
        'method': "pbe0-d3bj",
        'basis': 'def2-svp',
        'keywords': kw_id,
        'program': 'psi4'}}
ds_opt.add_specification(**add_spec,overwrite=True)
ds_opt.save()

In [ ]:
cr = ds_opt.compute(specification = 'b3lyp-d3_def2-svp', tag='refinement')
cr

In [28]:
kw = ptl.models.KeywordSet(**{"values": {"dftd": "d3", "convthre" : '3.0e-7', "threall" : '1.0e-13', 'dftgrid' : 2, "scf" : "diis+a"}})
kw_id = client.add_keywords([kw])[0]

add_ref_spec = {'name': 'pbe0-d3_def2-svp',
        'description': 'Geometric + TeraChem/pbe0-d3/def2-svp',
                'optimization_spec': {'program': 'geometric', 'keywords': {'converge' : ["set" , "gau_tight"], 'maxiter': 200}},
        'qc_spec': {'driver': 'gradient',
        'method': 'pbe0',
        'basis': 'def2-svp',
        'keywords': kw_id,
        'program': 'terachem'}}

ds_opt = client.get_collection("OptimizationDataset", sampling_collection_name)
ds_opt.add_specification(**add_ref_spec,overwrite=True)

In [29]:
cr = ds_opt.compute(specification = 'pbe0-d3_def2-svp', tag='refinement_tera')
cr

0

### 2) Restart Optimization Jobs 

In [ ]:
mol_list = []

spec = 'pbe0-d3_def2-svp'

for i in ds_opt.data.records: 
    #print(i)
    mol_list.append(i)

for i in  mol_list:
    opt_rec = ds_opt.get_record(name=i,specification= spec)
    if (opt_rec.status == 'ERROR'):    
        id_u = opt_rec.id 
        print('yes', i, id_u)
        c = client.modify_tasks("modify", new_tag = 'refinement_tera',base_result = id_u)
        #c = client.modify_tasks("restart",base_result = id_u)
        print(c)

### 3) Call errors from record

In [ ]:
print(rr.get_error().dict()['error_message'])

### 3) Delete Specification

In [66]:
#Delete Specification

#del ds_opt.data.specs['pbe0-d3bj_def2-svp']
#ds_opt.save()

### 4) Visualize Optimization for Molden

In [16]:
from pathlib import Path

tr_mols = rr.get_molecular_trajectory()
tr_grads = rr.get_trajectory()

molden_str = ''
for i in range(len(tr_mols)):
    enr = tr_grads[i].dict()['properties']['return_energy']
    molden_str += tr_mols[i].to_string(dtype='xyz').replace("0 1 "+tr_mols[i].get_molecular_formula(), str(enr))

opt_file = Path("./opt_trj_"+entry_name)

with opt_file.open("w", encoding ="utf-8") as f:
    f.write(molden_str)